In [167]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

df = pd.read_csv('data/merged_data.csv').drop(['strat_9','strat_14','strat_5','strat_4'], axis=1).dropna()


In [197]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

def select_columns_with_high_r_squared(df_returns, r_squared_threshold=0.8):
    results = {}
    columns = []
    strat_slope_map = {}

    for column in df_returns.columns:
        x = np.arange(len(df_returns))
        y = df_returns[column].cumsum()
        slope, intercept, r_value, p_value, std_err = linregress(x, y)
        
        results[column] = {
            'slope': slope,
            'intercept': intercept,
            'r_squared': r_value**2,
            'p_value': p_value,
            'std_err': std_err
        }
        
        if r_value**2 > r_squared_threshold:
            columns.append(column)
    
    for column in columns:
        slope_over_std = results[column]['slope'] / results[column]['std_err']
        strat_slope_map[column] = slope_over_std

    total_sum = sum(strat_slope_map.values())
    scaled_strat_slope_map = {strat: slope / total_sum for strat, slope in strat_slope_map.items()}

    return scaled_strat_slope_map

df_returns = df
output = select_columns_with_high_r_squared(df_returns)
print(output)


{'strat_7': 0.029089270856635117, 'strat_8': 0.04446405114123985, 'strat_10': -0.0640948294489768, 'strat_19': 0.029340891656754613, 'strat_21': 0.11824289003199435, 'strat_24': 0.041811437758140384, 'strat_25': 0.15740348528710307, 'strat_26': 0.03846526526197443, 'strat_27': 0.03541031527358192, 'strat_29': 0.04831569081960913, 'strat_30': 0.038411656488575224, 'strat_31': 0.0820484959672241, 'strat_33': -0.03213964137816105, 'strat_34': 0.038187551847570333, 'strat_37': 0.11917925848433794, 'strat_38': 0.16598251724915342, 'strat_40': 0.0327879783351551, 'strat_41': 0.0441240874295118, 'strat_42': 0.03296962693857686}


In [208]:
import pandas as pd
import numpy as np
import quantstats as qs

output = select_columns_with_high_r_squared(df_returns)

scaled_slope_over_std_filtered = np.clip(list(output.values()), -0.1, 0.1)

columns = list(output.keys())

returns = np.dot(scaled_slope_over_std_filtered, df_returns[columns].values.T)

cumulative_returns = pd.DataFrame(returns).cumsum()

sharpe_ratio = qs.stats.sharpe(pd.Series(returns))
print(sharpe_ratio)


1.8380927794306299


c:\Users\elias\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2742: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


In [210]:
import numpy as np

def adjust_weights(weights):
    lower_bound = -0.1
    upper_bound = 0.1
    weights = np.array(weights)
    adjusted_weights = np.where(weights > 0, upper_bound, lower_bound)
    current_sum = np.sum(adjusted_weights)
    scaling_factor = 1 / current_sum
    adjusted_weights *= scaling_factor
    return adjusted_weights

initial_weights = scaled_slope_over_std_filtered
adjusted_weights = adjust_weights(initial_weights)

print("Adjusted Weights:", adjusted_weights)
print("Sum of Adjusted Weights:", np.sum(adjusted_weights))


Adjusted Weights: [ 0.06666667  0.06666667 -0.06666667  0.06666667  0.06666667  0.06666667
  0.06666667  0.06666667  0.06666667  0.06666667  0.06666667  0.06666667
 -0.06666667  0.06666667  0.06666667  0.06666667  0.06666667  0.06666667
  0.06666667]
Sum of Adjusted Weights: 0.9999999999999998
